# Use AWS Glue Databrew from within Jupyter notebooks to prepare data for ML models 


---

This notebook  walks through the steps to configure and use  open source Jupyterlab extension for AWS Glue Databrew to prepare data for a sample anomaly detection model.

The [electricity consumption dataset](https://archive.ics.uci.edu/ml/datasets/ElectricityLoadDiagrams20112014#) is used in this notebook. A subset of original dataset with 4 customer datapoints is used as a starting point. A series of DataBrew transformations are applied on the dataset to prepare it for Random Cut Forests anomaly detection model.  On the prepared dataset, a RCF model is trained and deployed in SageMaker


Please make sure the kernel is set to 'python3'

Install the packages needed to run this notebook

In [ ]:
!pip install awswrangler
!pip install --upgrade sagemaker


#### Import the packages 

In [ ]:
import boto3
import sagemaker as sm
from sagemaker import *
import awswrangler as wr
import matplotlib.pyplot as plt
import os
import pandas as pd


#### S3 bucket where the raw and transformed data will be stored and the role details

In [ ]:
session = sm.Session()
### **** 'data_bucket' should point to bucket name you are using DataBrew and model Training ***** #### 
data_bucket=session.default_bucket() 
#s3_bucket=#input_s3_bucket#
role_arn=session.get_caller_identity_arn()


### Data Preparation using AWS Glue DataBrew

#### Exploring the prepared data

In [ ]:
pc_processed_path=os.path.join('s3://',data_bucket,'prefix_where_DataBrew_output_is_stored')
columns=['timestamp','client_id','hourly_consumption']
pc_processed_df = wr.s3.read_csv(path=pc_processed_path)
pc_processed_df=pc_processed_df [columns]
#columns[0]='timestamp'
#pc_processed_df.columns=columns
pc_processed_df.client_id.unique()


#### plotting the raw timeseries electricity consumption

In [ ]:
figure, axes = plt.subplots(3, 1)
figure.set_figheight(8)
figure.set_figwidth(15)
pc_processed_df[pc_processed_df['client_id']=='MT_012'].plot(ax=axes[0],title='MT_012')                                                
pc_processed_df[pc_processed_df['client_id']=='MT_013'].plot(ax=axes[1],title='MT_013')
pc_processed_df[pc_processed_df['client_id']=='MT_132'].plot(ax=axes[2],title='MT_132')

#### Lets train our model with ***MT_132*** consumption data. Since RCF requires one time series and integer values, lets filter and convert the consumption data to inetger data type

In [ ]:
train_df=pc_processed_df[(pc_processed_df['client_id']=='MT_132') & (pc_processed_df['timestamp']<'2014-11-01')]
train_df=train_df.drop(['timestamp','client_id'],axis=1)
train_df.hourly_consumption=train_df.hourly_consumption.astype('int32')
train_df.head()

### Train RCF Model

In [ ]:
s3_train_path=os.path.join('s3://',data_bucket,'databrew_rcf','training','train.csv')
s3_model_path=os.path.join('s3://',data_bucket,'databrew_rcf','model')

wr.s3.to_csv(df=train_df,path=s3_train_path,header=False,index=False)
training_channel=sm.inputs.TrainingInput(s3_data=s3_train_path,content_type='text/csv;label_size=0',distribution='ShardedByS3Key')
channels={'train':training_channel}


In [ ]:
rcf_algo_uri=image_uris.retrieve('randomcutforest',session.boto_region_name)
rcf_estimator= sm.estimator.Estimator(rcf_algo_uri,role=role_arn,instance_count=1,instance_type='ml.m5.large',output_path=s3_model_path)
rcf_estimator.set_hyperparameters(feature_dim=1)
rcf_estimator.fit(channels)
                            

### Deploy the trained model 

In [ ]:
rcf_endpoint_name='databrew-rcf-demo-endpoint'
rcf_predictor=rcf_estimator.deploy(endpoint_name=rcf_endpoint_name,instance_type='ml.t2.medium',initial_instance_count=1,serializer=serializers.CSVSerializer(),deserializer=deserializers.JSONDeserializer())


### Predictions and Cleanup

In [ ]:
from statistics import mean,stdev
test_df=pc_processed_df[(pc_processed_df['client_id']=='MT_012') & (pc_processed_df['timestamp'] >= '2014-01-01') &(pc_processed_df['hourly_consumption'] != 0)]
test_df=test_df.tail(500)
test_df_values=test_df['hourly_consumption'].astype('str').tolist()
response=rcf_predictor.predict(test_df_values)
scores = [datum["score"] for datum in response["scores"]]
scores_mean=mean(scores)
scores_std=stdev(scores)


### plot the prediction scores taking mean + or - 2*standard_deviation as the baseline  

In [ ]:
test_df['hourly_consumption'].plot(figsize=(40,10))

In [ ]:
plt.figure(figsize=(40,10))
plt.plot(scores)
plt.autoscale(tight=True)
plt.axhline(y=scores_mean+2*scores_std,color='red')
plt.show()

### Clean up by deleting the endpoint

In [ ]:
rcf_predictor.delete_endpoint()